In [1]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

There is also a CSV at https://data.cityofchicago.org/Health-Human-Services/Daily-COVID-19-Cases-and-Deaths/kxzd-kd6a, but this appears to be based on reported dates for deaths, which is not clean.

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [26]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 2,
                                   22: 2, 23: 2, 24: 2, 25: 5, 26: 2,
                                   27: 8, 28: 4, 29: 21, 30: 12,
                                   31: 16, 32: 24, 33: 14, 34: 24,
                                   35: 21, 36: 26, 37: 22, 38: 25,
                                   39: 34, 40: 25, 41: 31, 42: 32,
                                   43: 33, 44: 31, 45: 34, 46: 31,
                                   47: 30, 48: 40, 49: 37, 50: 35,
                                   51: 37, 52: 34, 53: 45, 54: 34,
                                   55: 50, 56: 40, 57: 48, 58: 36,
                                   59: 55, 60: 40, 61: 37, 62: 38,
                                   63: 49, 64: 42, 65: 48, 66: 56,
                                   67: 44, 68: 48, 69: 43, 70: 50,
                                   71: 37, 72: 40, 73: 47, 74: 41,
                                   75: 41, 76: 49, 77: 29, 78: 38,
                                   79: 32, 80: 27, 81: 28, 82: 33,
                                   83: 34, 84: 26, 85: 23, 86: 21,
                                   87: 17, 88: 23, 89: 16, 90: 12,
                                   91: 17, 92: 10})

In [27]:
chi_last_day = max(chi_death_dict.keys())

In [28]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [29]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [30]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-05-27,23
2020-05-28,16
2020-05-29,12
2020-05-30,17
2020-05-31,10


In [31]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')